# MRI pRF processing
Perform these steps after recombining the partial result-volumes that are returned by the cluster. <br>
Either use: <br> 
`../PostFit/matlab/ck_Combine_MRI_SliceChunks_cv.m`, <br> 
or use a batch operation with: <br> 
`../PostFit/matlab/BATCH_ck_Combine_MRI_SliceChunks.m`.  <br> 

If you have done that, there should be folders: <br> 
`../FitResults/MRI/<monkey>/<model>/` <br> 
with sets of results volumes: <br> 
`../FitResults/MRI/<monkey>/<model>/Sess-<model>.nii.gz`       

## Configure path and inclusion criteria

In [1]:
FitRes=/Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults
ManMask=/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks

# declare some arrays to cycle over 
declare -a Monkeys=('danny' 'eddy')
declare -a Models=(\
    'csshrf_cv1_dhrf'
    'csshrf_cv1_mhrf'
    'doghrf_cv1_dhrf'
    'doghrf_cv1_mhrf'
    'linhrf_cv1_dhrf'
    'linhrf_cv1_mhrf'
    'linhrf_cv1_dhrf_neggain'
    'linhrf_cv1_mhrf_neggain')

## Define a function to process

In [2]:
function MaskResults() {
    MONKEY=$1
    sess=$2
    RTH=$3
    FitRes=$4
    ManMask=$5
    
    wp=$FitRes/MRI/$MONKEY/$sess
    
    brainmask=$ManMask/sub-${MONKEY}/func/sub-${MONKEY}_ref_func_mask_res-1x1x1.nii.gz
    echo "Masking results for" $MONKEY $sess "at mean xval R2 value of" $RTH

    # Create the appropriate masks
    fslmaths ${brainmask} -bin $wp/brainmask.nii.gz
    fslmaths $wp/brainmask.nii.gz -sub 1 -mul -1 $wp/nonbrainmask.nii.gz
    fslmaths $wp/nonbrainmask.nii.gz -mul -99 $wp/nanreplacement.nii.gz

    if [ ! -d "$wp/org" ]; then
        mkdir -p $wp/org
        cp $wp/Sess-*.nii.gz $wp/org/
    else
        cp $wp/org/Sess-*.nii.gz $wp/
    fi
    
    # create average/max/min R2 values
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -nan $wp/'Sess-'$sess'_R2_1.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_2.nii.gz' -nan $wp/'Sess-'$sess'_R2_2.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -min $wp/'Sess-'$sess'_R2_2.nii.gz' $wp/'Sess-'$sess'_minR2.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -max $wp/'Sess-'$sess'_R2_2.nii.gz' $wp/'Sess-'$sess'_maxR2.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -add $wp/'Sess-'$sess'_R2_2.nii.gz' -div 2 $wp/'Sess-'$sess'_meanR2.nii.gz'

    # create a mask volume based on mean R2
    fslmaths $wp/'Sess-'$sess'_meanR2.nii.gz' -thr $RTH -bin $wp/'mask_th'$RTH'.nii.gz'
    fslmaths $wp/'mask_th'$RTH'.nii.gz' -sub 1 -mul -1 $wp/'invmask_th'$RTH'.nii.gz'
    fslmaths $wp/'mask_th'$RTH'.nii.gz' -sub 1 -mul 99 $wp/'negvalmask_th'$RTH'.nii.gz'

    # mask results
    for part in 1 2; do
        # echo 'Part ' ${part}
        fslmaths $wp/'Sess-'$sess'_ang_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'ANG_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_ecc_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'ECC_'${part}'_th'$RTH'.nii.gz'

        if [ ${sess:0:3} = css ]; then
            fslmaths $wp/'Sess-'$sess'_expt_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'EXPT_'${part}'_th'$RTH'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_exptsq_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'EXPTSQ_'${part}'_th'$RTH'.nii.gz'
        fi

        fslmaths $wp/'Sess-'$sess'_imag_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'IMAG_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_real_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'REAL_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_FWHM_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'FWHM_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_rfsize_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'RFS_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_gain_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'GAIN_'${part}'_th'$RTH'.nii.gz'

        if [ ${sess:0:3} = dog ]; then
            fslmaths $wp/'Sess-'$sess'_normamp_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'NAMP_'${part}'_th'$RTH'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sd2_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'IRFS_'${part}'_th'$RTH'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sdratio_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'SDRATIO_'${part}'_th'$RTH'.nii.gz'
        fi

        # ======

        fslmaths $wp/'Sess-'$sess'_ang_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz $wp/'Sess-'$sess'_ang_'${part}'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_ecc_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz $wp/'Sess-'$sess'_ecc_'${part}'.nii.gz'

        if [ ${sess:0:3} = css ]; then
            fslmaths $wp/'Sess-'$sess'_expt_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_expt_'${part}'.nii.gz' 
            fslmaths $wp/'Sess-'$sess'_exptsq_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_exptsq_'${part}'.nii.gz' 
        fi

        fslmaths $wp/'Sess-'$sess'_imag_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_imag_'${part}'.nii.gz' 
        fslmaths $wp/'Sess-'$sess'_real_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_real_'${part}'.nii.gz' 
        fslmaths $wp/'Sess-'$sess'_FWHM_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_FWHM_'${part}'.nii.gz'  
        fslmaths $wp/'Sess-'$sess'_rfsize_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_rfsize_'${part}'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_gain_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_gain_'${part}'.nii.gz'

        if [ ${sess:0:3} = dog ]; then
            fslmaths $wp/'Sess-'$sess'_normamp_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_normamp_'${part}'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sd2_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_sd2_'${part}'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sdratio_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_sdratio_'${part}'.nii.gz'
        fi

        mkdir -p $wp/TH_$RTH
        mv $wp/'ANG_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'ECC_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/

        if [ ${sess:0:3} = css ]; then
            mv $wp/'EXPT_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
            mv $wp/'EXPTSQ_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        fi

        mv $wp/'IMAG_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'REAL_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'FWHM_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'RFS_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'GAIN_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/

        if [ ${sess:0:3} = dog ]; then
            mv $wp/'NAMP_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
            mv $wp/'IRFS_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
            mv $wp/'SDRATIO_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        fi    
    done
    mv $wp/'mask_th'$RTH'.nii.gz' $wp/TH_$RTH/
    mv $wp/'invmask_th'$RTH'.nii.gz' $wp/TH_$RTH/
    mv $wp/'negvalmask_th'$RTH'.nii.gz' $wp/TH_$RTH/
    mv $wp/nanreplacement.nii.gz $wp/TH_$RTH/

    wpth=$wp/TH_$RTH
        
    fslmaths $wpth/'FWHM_1_th'$RTH'.nii.gz' -add $wpth/'FWHM_2_th'$RTH'.nii.gz' -div 2 $wpth/'FWHM_th'$RTH'.nii.gz'
    fslmaths $wpth/'RFS_1_th'$RTH'.nii.gz' -add $wpth/'RFS_2_th'$RTH'.nii.gz' -div 2 $wpth/'RFS_th'$RTH'.nii.gz'
    fslmaths $wpth/'GAIN_1_th'$RTH'.nii.gz' -add $wpth/'GAIN_2_th'$RTH'.nii.gz' -div 2 $wpth/'GAIN_th'$RTH'.nii.gz'
    fslmaths $wpth/'REAL_1_th'$RTH'.nii.gz' -add $wpth/'REAL_2_th'$RTH'.nii.gz' -div 2 $wpth/'REAL_th'$RTH'.nii.gz'
    fslmaths $wpth/'IMAG_1_th'$RTH'.nii.gz' -add $wpth/'IMAG_2_th'$RTH'.nii.gz' -div 2 $wpth/'IMAG_th'$RTH'.nii.gz'
    
    if [ ${sess:0:3} = css ]; then
        fslmaths $wpth/'EXPT_1_th'$RTH'.nii.gz' -add $wpth/'EXPT_2_th'$RTH'.nii.gz' -div 2 $wpth/'EXPT_th'$RTH'.nii.gz'
        fslmaths $wpth/'EXPTSQ_1_th'$RTH'.nii.gz' -add $wpth/'EXPTSQ_2_th'$RTH'.nii.gz' -div 2 $wpth/'EXPTSQ_th'$RTH'.nii.gz'
    fi
    if [ ${sess:0:3} = dog ]; then
        fslmaths $wpth/'NAMP_1_th'$RTH'.nii.gz' -add $wpth/'NAMP_2_th'$RTH'.nii.gz' -div 2 $wpth/'NAMP_th'$RTH'.nii.gz'
        fslmaths $wpth/'IRFS_1_th'$RTH'.nii.gz' -add $wpth/'IRFS_2_th'$RTH'.nii.gz' -div 2 $wpth/'IRFS_th'$RTH'.nii.gz'
        fslmaths $wpth/'SDRATIO_1_th'$RTH'.nii.gz' -add $wpth/'SDRATIO_2_th'$RTH'.nii.gz' -div 2 $wpth/'SDRATIO_th'$RTH'.nii.gz'
    fi

    for xv in 1 2; do
        fslmaths $wpth/'ANG_'${xv}'_th'$RTH'.nii.gz' -mul 0.0174533 -cos -mul \
            $wpth/'ECC_'${xv}'_th'$RTH'.nii.gz' $wpth/'X_'${xv}'_th'$RTH'.nii.gz'
        fslmaths $wpth/'ANG_'${xv}'_th'$RTH'.nii.gz' -mul 0.0174533 -sin -mul \
            $wpth/'ECC_'${xv}'_th'$RTH'.nii.gz' $wpth/'Y_'${xv}'_th'$RTH'.nii.gz'
    done

    fslmaths $wpth/'X_1_th'$RTH'.nii.gz' -add $wpth/'X_2_th'$RTH'.nii.gz' -div 2 -mas \
        $wpth/'mask_th'$RTH'.nii.gz' -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'X_th'$RTH'.nii.gz'
    fslmaths $wpth/'Y_1_th'$RTH'.nii.gz' -add $wpth/'Y_2_th'$RTH'.nii.gz' -div 2 -mas \
        $wpth/'mask_th'$RTH'.nii.gz' -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'Y_th'$RTH'.nii.gz'

    fslmaths $wpth/'X_th'$RTH'.nii.gz' -sqr $wpth/'X2_th'$RTH'.nii.gz'
    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -sqr $wpth/'Y2_th'$RTH'.nii.gz'
    fslmaths $wpth/'X2_th'$RTH'.nii.gz' -add $wpth/'Y2_th'$RTH'.nii.gz' -sqrt -mas $wpth/'mask_th'$RTH'.nii.gz' \
        -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'ECC_th'$RTH'.nii.gz'

    # calculate angle polar to cartesian
    fslmaths $wpth/'X_th'$RTH'.nii.gz' -binv -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negX.nii.gz'
    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -binv -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negY.nii.gz'
    fslmaths $wpth/'X_th'$RTH'.nii.gz' -bin -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'posX.nii.gz'
    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -bin -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'posY.nii.gz'
    fslmaths $wpth/'negX.nii.gz' -mul $wpth/'posY.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negXposY.nii.gz'
    fslmaths $wpth/'negX.nii.gz' -mul $wpth/'negY.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negXnegY.nii.gz'

    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -div $wpth/'X_th'$RTH'.nii.gz' -atan -mul 57.2957795 $wpth/'rawANG.nii.gz'
    # +X +Y  >>  atan(Y/X)
    # do nothing
    # -X -Y  >>  atan(Y/X) -180
    fslmaths $wpth/'negXnegY.nii.gz' -mul -180 -add $wpth/'rawANG.nii.gz' $wpth/'raw2ANG.nii.gz'
    # -X +Y  >>  atan(Y/X) +180
    fslmaths $wpth/'negXposY.nii.gz' -mul 180 -add $wpth/'raw2ANG.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' \
        -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'raw3ANG.nii.gz'
    fslmaths $wpth/'raw3ANG.nii.gz' -binv -mul 360 -add $wpth/'raw3ANG.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' \
        -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'ANG_th'$RTH'.nii.gz'

    # clean up, move some intermediate results to a separate folder
    mkdir -p $wpth/'cv_r1' 
    mv $wpth/*_1_*.nii.gz $wpth/cv_r1/
    mkdir -p $wpth/'cv_r2'
    mv $wpth/*_2_*.nii.gz $wpth/cv_r2/
    mkdir -p $wpth/'helper_vols'
    mv $wpth/*pos*.nii.gz $wpth/helper_vols/
    mv $wpth/*neg*.nii.gz $wpth/helper_vols/
    mv $wpth/*raw*.nii.gz $wpth/helper_vols/
    mv $wpth/*X2*.nii.gz $wpth/helper_vols/
    mv $wpth/*Y2*.nii.gz $wpth/helper_vols/

}

In [3]:
function MaskResults_inAnat() {
    MONKEY=$1
    sess=$2
    RTH=$3
    FitRes=$4
    ManMask=$5
    
    wp=$FitRes/MRI/$MONKEY/$sess
    mkdir -p $wp/inAnat
    
    cp $ManMask/sub-${MONKEY}/anat/sub-${MONKEY}_ref* $wp/inAnat/
    cp $ManMask/sub-${MONKEY}/warps/sub-${MONKEY}_func2* $wp/inAnat/
    
    for f in $wp/org/Sess-*.nii.gz; do
        # applywarp fun2anat
        f2=`basename $f .nii.gz`
        flirt -applyxfm -init $wp/inAnat/sub-${MONKEY}_func2anat.mat -in $f \
            -ref $wp/inAnat/sub-${MONKEY}_ref_anat_res-1x1x1.nii.gz \
            -out $wp/inAnat/${f2}.nii -interp nearestneighbour
    done
    wp=$wp/inAnat
    
    brainmask=$ManMask/sub-${MONKEY}/anat/sub-${MONKEY}_ref_anat_mask_res-1x1x1.nii.gz

    echo "Masking results for" $MONKEY $sess "at mean xval R2 value of" $RTH

    # Create the appropriate masks
    fslmaths ${brainmask} -bin $wp/brainmask.nii.gz
    fslmaths $wp/brainmask.nii.gz -sub 1 -mul -1 $wp/nonbrainmask.nii.gz
    fslmaths $wp/nonbrainmask.nii.gz -mul -99 $wp/nanreplacement.nii.gz

    if [ ! -d "$wp/org" ]; then
        mkdir -p $wp/org
        cp $wp/Sess-*.nii.gz $wp/org/
    fi
    
    # create average/max/min R2 values
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -nan $wp/'Sess-'$sess'_R2_1.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_2.nii.gz' -nan $wp/'Sess-'$sess'_R2_2.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -min $wp/'Sess-'$sess'_R2_2.nii.gz' $wp/'Sess-'$sess'_minR2.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -max $wp/'Sess-'$sess'_R2_2.nii.gz' $wp/'Sess-'$sess'_maxR2.nii.gz'
    fslmaths $wp/'Sess-'$sess'_R2_1.nii.gz' -add $wp/'Sess-'$sess'_R2_2.nii.gz' -div 2 $wp/'Sess-'$sess'_meanR2.nii.gz'

    # create a mask volume based on mean R2
    fslmaths $wp/'Sess-'$sess'_meanR2.nii.gz' -thr $RTH -bin $wp/'mask_th'$RTH'.nii.gz'
    fslmaths $wp/'mask_th'$RTH'.nii.gz' -sub 1 -mul -1 $wp/'invmask_th'$RTH'.nii.gz'
    # apply brainmask as well
    fslmaths $wp/'mask_th'$RTH'.nii.gz' -mas $wp/brainmask.nii.gz $wp/'mask_th'$RTH'.nii.gz'
    fslmaths $wp/'invmask_th'$RTH'.nii.gz' -mas $wp/brainmask.nii.gz $wp/'invmask_th'$RTH'.nii.gz'
    fslmaths $wp/'mask_th'$RTH'.nii.gz' -sub 1 -mul 99 $wp/'negvalmask_th'$RTH'.nii.gz'
    
    # mask results
    for part in 1 2; do
        # echo 'Part ' ${part}
        fslmaths $wp/'Sess-'$sess'_ang_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'ANG_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_ecc_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'ECC_'${part}'_th'$RTH'.nii.gz'

        if [ ${sess:0:3} = css ]; then
            fslmaths $wp/'Sess-'$sess'_expt_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'EXPT_'${part}'_th'$RTH'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_exptsq_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'EXPTSQ_'${part}'_th'$RTH'.nii.gz'
        fi

        fslmaths $wp/'Sess-'$sess'_imag_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'IMAG_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_real_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'REAL_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_FWHM_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'FWHM_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_rfsize_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'RFS_'${part}'_th'$RTH'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_gain_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
            -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'GAIN_'${part}'_th'$RTH'.nii.gz'

        if [ ${sess:0:3} = dog ]; then
            fslmaths $wp/'Sess-'$sess'_normamp_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'NAMP_'${part}'_th'$RTH'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sd2_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -div 10 -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'IRFS_'${part}'_th'$RTH'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sdratio_'${part}'.nii.gz' -nan -mas $wp/'mask_th'$RTH'.nii.gz' \
                -add $wp/'negvalmask_th'$RTH'.nii.gz' $wp/'SDRATIO_'${part}'_th'$RTH'.nii.gz'
        fi

        # ======

        fslmaths $wp/'Sess-'$sess'_ang_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz $wp/'Sess-'$sess'_ang_'${part}'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_ecc_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz $wp/'Sess-'$sess'_ecc_'${part}'.nii.gz'

        if [ ${sess:0:3} = css ]; then
            fslmaths $wp/'Sess-'$sess'_expt_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_expt_'${part}'.nii.gz' 
            fslmaths $wp/'Sess-'$sess'_exptsq_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_exptsq_'${part}'.nii.gz' 
        fi

        fslmaths $wp/'Sess-'$sess'_imag_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_imag_'${part}'.nii.gz' 
        fslmaths $wp/'Sess-'$sess'_real_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_real_'${part}'.nii.gz' 
        fslmaths $wp/'Sess-'$sess'_FWHM_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_FWHM_'${part}'.nii.gz'  
        fslmaths $wp/'Sess-'$sess'_rfsize_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_rfsize_'${part}'.nii.gz'
        fslmaths $wp/'Sess-'$sess'_gain_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
            $wp/'Sess-'$sess'_gain_'${part}'.nii.gz'

        if [ ${sess:0:3} = dog ]; then
            fslmaths $wp/'Sess-'$sess'_normamp_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_normamp_'${part}'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sd2_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_sd2_'${part}'.nii.gz'
            fslmaths $wp/'Sess-'$sess'_sdratio_'${part}'.nii.gz' -nan -add $wp/nanreplacement.nii.gz \
                $wp/'Sess-'$sess'_sdratio_'${part}'.nii.gz'
        fi

        mkdir -p $wp/TH_$RTH
        mv $wp/'ANG_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'ECC_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/

        if [ ${sess:0:3} = css ]; then
            mv $wp/'EXPT_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
            mv $wp/'EXPTSQ_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        fi

        mv $wp/'IMAG_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'REAL_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'FWHM_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'RFS_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        mv $wp/'GAIN_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/

        if [ ${sess:0:3} = dog ]; then
            mv $wp/'NAMP_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
            mv $wp/'IRFS_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
            mv $wp/'SDRATIO_'${part}'_th'$RTH'.nii.gz' $wp/TH_$RTH/
        fi    
    done
    mv $wp/'mask_th'$RTH'.nii.gz' $wp/TH_$RTH/
    mv $wp/'invmask_th'$RTH'.nii.gz' $wp/TH_$RTH/
    mv $wp/'negvalmask_th'$RTH'.nii.gz' $wp/TH_$RTH/
    mv $wp/nanreplacement.nii.gz $wp/TH_$RTH/

    wpth=$wp/TH_$RTH
        
    fslmaths $wpth/'FWHM_1_th'$RTH'.nii.gz' -add $wpth/'FWHM_2_th'$RTH'.nii.gz' -div 2 $wpth/'FWHM_th'$RTH'.nii.gz'
    fslmaths $wpth/'RFS_1_th'$RTH'.nii.gz' -add $wpth/'RFS_2_th'$RTH'.nii.gz' -div 2 $wpth/'RFS_th'$RTH'.nii.gz'
    fslmaths $wpth/'GAIN_1_th'$RTH'.nii.gz' -add $wpth/'GAIN_2_th'$RTH'.nii.gz' -div 2 $wpth/'GAIN_th'$RTH'.nii.gz'
    fslmaths $wpth/'REAL_1_th'$RTH'.nii.gz' -add $wpth/'REAL_2_th'$RTH'.nii.gz' -div 2 $wpth/'REAL_th'$RTH'.nii.gz'
    fslmaths $wpth/'IMAG_1_th'$RTH'.nii.gz' -add $wpth/'IMAG_2_th'$RTH'.nii.gz' -div 2 $wpth/'IMAG_th'$RTH'.nii.gz'
    
    if [ ${sess:0:3} = css ]; then
        fslmaths $wpth/'EXPT_1_th'$RTH'.nii.gz' -add $wpth/'EXPT_2_th'$RTH'.nii.gz' -div 2 $wpth/'EXPT_th'$RTH'.nii.gz'
        fslmaths $wpth/'EXPTSQ_1_th'$RTH'.nii.gz' -add $wpth/'EXPTSQ_2_th'$RTH'.nii.gz' -div 2 $wpth/'EXPTSQ_th'$RTH'.nii.gz'
    fi
    if [ ${sess:0:3} = dog ]; then
        fslmaths $wpth/'NAMP_1_th'$RTH'.nii.gz' -add $wpth/'NAMP_2_th'$RTH'.nii.gz' -div 2 $wpth/'NAMP_th'$RTH'.nii.gz'
        fslmaths $wpth/'IRFS_1_th'$RTH'.nii.gz' -add $wpth/'IRFS_2_th'$RTH'.nii.gz' -div 2 $wpth/'IRFS_th'$RTH'.nii.gz'
        fslmaths $wpth/'SDRATIO_1_th'$RTH'.nii.gz' -add $wpth/'SDRATIO_2_th'$RTH'.nii.gz' -div 2 $wpth/'SDRATIO_th'$RTH'.nii.gz'
    fi

    for xv in 1 2; do
        fslmaths $wpth/'ANG_'${xv}'_th'$RTH'.nii.gz' -mul 0.0174533 -cos -mul \
            $wpth/'ECC_'${xv}'_th'$RTH'.nii.gz' $wpth/'X_'${xv}'_th'$RTH'.nii.gz'
        fslmaths $wpth/'ANG_'${xv}'_th'$RTH'.nii.gz' -mul 0.0174533 -sin -mul \
            $wpth/'ECC_'${xv}'_th'$RTH'.nii.gz' $wpth/'Y_'${xv}'_th'$RTH'.nii.gz'
    done

    fslmaths $wpth/'X_1_th'$RTH'.nii.gz' -add $wpth/'X_2_th'$RTH'.nii.gz' -div 2 -mas \
        $wpth/'mask_th'$RTH'.nii.gz' -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'X_th'$RTH'.nii.gz'
    fslmaths $wpth/'Y_1_th'$RTH'.nii.gz' -add $wpth/'Y_2_th'$RTH'.nii.gz' -div 2 -mas \
        $wpth/'mask_th'$RTH'.nii.gz' -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'Y_th'$RTH'.nii.gz'

    fslmaths $wpth/'X_th'$RTH'.nii.gz' -sqr $wpth/'X2_th'$RTH'.nii.gz'
    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -sqr $wpth/'Y2_th'$RTH'.nii.gz'
    fslmaths $wpth/'X2_th'$RTH'.nii.gz' -add $wpth/'Y2_th'$RTH'.nii.gz' -sqrt -mas $wpth/'mask_th'$RTH'.nii.gz' \
        -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'ECC_th'$RTH'.nii.gz'

    # calculate angle polar to cartesian
    fslmaths $wpth/'X_th'$RTH'.nii.gz' -binv -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negX.nii.gz'
    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -binv -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negY.nii.gz'
    fslmaths $wpth/'X_th'$RTH'.nii.gz' -bin -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'posX.nii.gz'
    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -bin -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'posY.nii.gz'
    fslmaths $wpth/'negX.nii.gz' -mul $wpth/'posY.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negXposY.nii.gz'
    fslmaths $wpth/'negX.nii.gz' -mul $wpth/'negY.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' $wpth/'negXnegY.nii.gz'

    fslmaths $wpth/'Y_th'$RTH'.nii.gz' -div $wpth/'X_th'$RTH'.nii.gz' -atan -mul 57.2957795 $wpth/'rawANG.nii.gz'
    # +X +Y  >>  atan(Y/X)
    # do nothing
    # -X -Y  >>  atan(Y/X) -180
    fslmaths $wpth/'negXnegY.nii.gz' -mul -180 -add $wpth/'rawANG.nii.gz' $wpth/'raw2ANG.nii.gz'
    # -X +Y  >>  atan(Y/X) +180
    fslmaths $wpth/'negXposY.nii.gz' -mul 180 -add $wpth/'raw2ANG.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' \
        -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'raw3ANG.nii.gz'
    fslmaths $wpth/'raw3ANG.nii.gz' -binv -mul 360 -add $wpth/'raw3ANG.nii.gz' -mas $wpth/'mask_th'$RTH'.nii.gz' \
        -add $wpth/'negvalmask_th'$RTH'.nii.gz' $wpth/'ANG_th'$RTH'.nii.gz'

    # clean up, move some intermediate results to a separate folder
    mkdir -p $wpth/'cv_r1' 
    mv $wpth/*_1_*.nii.gz $wpth/cv_r1/
    mkdir -p $wpth/'cv_r2'
    mv $wpth/*_2_*.nii.gz $wpth/cv_r2/
    mkdir -p $wpth/'helper_vols'
    mv $wpth/*pos*.nii.gz $wpth/helper_vols/
    mv $wpth/*neg*.nii.gz $wpth/helper_vols/
    mv $wpth/*raw*.nii.gz $wpth/helper_vols/
    mv $wpth/*X2*.nii.gz $wpth/helper_vols/
    mv $wpth/*Y2*.nii.gz $wpth/helper_vols/

}

## Run the masking function for all defined cases

In [4]:
# cycle over monkeys, models, and R2-thresholds
for M in "${Monkeys[@]}"; do
    for S in "${Models[@]}"; do
        for rth in 0 1 2 3 4 5 10; do
             MaskResults $M $S $rth $FitRes $ManMask
        done
    done
done        

Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 0
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 1
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 2
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 3
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 4
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 5
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 10
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 0
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 1
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 2
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 3
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 4
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 5
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 10
Masking results for danny doghrf

## Do it again with everything in anatomical space
Because we may want to look in volumetric space projected on anatomy

In [5]:
# cycle over monkeys, models, and R2-thresholds
for M in "${Monkeys[@]}"; do
    for S in "${Models[@]}"; do
        for rth in 0 1 2 3 4 5 10; do
             MaskResults_inAnat $M $S $rth $FitRes $ManMask
        done
    done
done    

Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 0
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 1
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 2
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 3
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 4
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 5
Masking results for danny csshrf_cv1_dhrf at mean xval R2 value of 10
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 0
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 1
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 2
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 3
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 4
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 5
Masking results for danny csshrf_cv1_mhrf at mean xval R2 value of 10
Masking results for danny doghrf

# Copy reference files to separate reference folders

In [ ]:
for M in "${Monkeys[@]}"; do
    mkdir -p ${FitRes}/Reference/Volumes/func/${M}
    cp -r ${ManMask}/sub-${M}/func/* ${FitRes}/Reference/Volumes/func/${M}/
    mkdir -p ${FitRes}/Reference/Volumes/anat/${M}
    cp -r ${ManMask}/sub-${M}/anat/* ${FitRes}/Reference/Volumes/anat/${M}/
    mkdir -p ${FitRes}/Reference/Volumes/warps/${M}
    cp -r ${ManMask}/sub-${M}/warps/* ${FitRes}/Reference/Volumes/warps/${M}/
done

# Copy atlas files to separate reference folders
Within the `manual-masks` folder there is a notebook `Atlas2ManualMasks` that will export the atlas data (D99) from the NMT template folders to the manual-masks folder. After that we can continue here for additional processing.

In [ ]:
for M in "${Monkeys[@]}"; do
    mkdir -p ${FitRes}/Reference/Volumes/atlas/${M}
    cp -r ${ManMask}/sub-${M}/atlas/* ${FitRes}/Reference/Volumes/atlas/${M}/
    # rm -r ${FitRes}/Reference/Volumes/atlas/${M}/atlas
done

### Adjust ROIs & warp to functional space
Do this in `manual-masks` so it will be generic. See the notebook `Customize_atlasses` in the `manual-masks/notebooks` folder.   

# Mask results with ROI    
This will take the results in anatomical space and mask based on ROI so we can easily display

In [6]:
function MaskResults_inAnat_withROI () {
    MONKEY=$1
    MODEL=$2
    ROI=$3
    FitRes=$4

    echo "Processing monkey ${MONKEY}"
    echo "${MODEL} ${ROI}" 

    M_FLD=${FitRes}/MRI/${MONKEY}/${MODEL}/inAnat
    ROI_FLD=${FitRes}/Reference/Volumes/atlas/${MONKEY}/ROI_manualadjust

    # do this for all existing thresholded subfolders 
    for sf in ${M_FLD}/TH_*; do
        if [ -d ${sf} ]; then
            #echo Processing $sf
            
            # mask averaged results with these ROIs
            # NB! Use the filenames from ROI_manualadjust
            # use specified ROI or this list

            if [ ${ROI} = 'allrois' ]; then 
                declare -a arr=(
                    "1-2"
                    "3a_b"
                    "5_(PE)"
                    "5_(PEa)"
                    "5_(PEc)"
                    "7a_(Opt_PG)"
                    "7b_(PFG_PF)"
                    "7m"
                    "7op"
                    "8Ad"
                    "8Av"
                    "8Bd"
                    "8Bm"
                    "8Bs"
                    "9d"
                    "9m"
                    "10mc"
                    "10mr"
                    "10o"
                    "11l"
                    "11m"
                    "12l"
                    "12m"
                    "12o"
                    "12r"
                    "13a"
                    "13b"
                    "13l"
                    "13m"
                    "14c"
                    "14r"
                    "23a"
                    "23b"
                    "23c"
                    "24a"
                    "24a'"
                    "24b"
                    "24b'"
                    "24c"
                    "24c'"
                    "25"
                    "29"
                    "30"
                    "31"
                    "32"
                    "35"
                    "36c"
                    "36p"
                    "36r"
                    "44"
                    "45a"
                    "45b"
                    "46d"
                    "46f"
                    "46v"
                    "A1"
                    "AAA"
                    "ABmc"
                    "ABpc"
                    "AHA"
                    "AIP"
                    "AL"
                    "AONd_m"
                    "AONl"
                    "Bi"
                    "Bmc"
                    "Bpc"
                    "CA1"
                    "CA2"
                    "CA3"
                    "CA4"
                    "CE"
                    "Cerebellum"
                    "CL"
                    "Claustrum"
                    "CM"
                    "COa"
                    "COp"
                    "CPB"
                    "DG"
                    "DP"
                    "EC"
                    "ECL"
                    "EI"
                    "ELc"
                    "ELr"
                    "EO"
                    "ER"
                    "F1_(4)"
                    "F2_(6DR_6DC)"
                    "F3"
                    "F4"
                    "F5_(6Va_6Vb)"
                    "F6"
                    "F7"
                    "FST"
                    "G"
                    "GPe_GPi"
                    "Ia"
                    "Iai"
                    "Ial"
                    "Iam"
                    "Iapl"
                    "Iapm"
                    "Id"
                    "Ig"
                    "IPa"
                    "Ld"
                    "LGNm"
                    "LGNp"
                    "LIPd"
                    "LIPv"
                    "LOP"
                    "Lv"
                    "MDdc"
                    "MDpc"
                    "ME"
                    "MGad"
                    "MGmc"
                    "MGpd"
                    "MGv"
                    "MGz"
                    "MIP"
                    "ML"
                    "MN"
                    "MST"
                    "MT"
                    "NLOT"
                    "OT"
                    "PAC2"
                    "PAC3"
                    "PACo_Pir"
                    "PACs"
                    "paraS"
                    "PEci"
                    "PGa"
                    "Pi"
                    "PIP"
                    "Pir"
                    "PL"
                    "PL'"
                    "PM"
                    "PrCO"
                    "preS"
                    "proS"
                    "R"
                    "RM"
                    "RPB"
                    "RT"
                    "RTL"
                    "RTp"
                    "SII"
                    "SNr_c"
                    "STGr"
                    "STN"
                    "Striatum"
                    "Sub"
                    "TAa"
                    "TEa"
                    "TEad"
                    "TEav"
                    "TEm"
                    "TEO"
                    "TEpd"
                    "TEpv"
                    "TF"
                    "TFO"
                    "TGa"
                    "TGdd"
                    "TGdg"
                    "TGsts"
                    "TGvd"
                    "TGvg"
                    "TH"
                    "TPO"
                    "Tpt"
                    "TTv"
                    "v23a"
                    "v23b"
                    "v23b?"
                    "V1"
                    "V2"
                    "V2?"
                    "V2_or_v23b"
                    "V3A"
                    "V3d"
                    "V3v"
                    "V4"
                    "V4t"
                    "V4v"
                    "V6"
                    "V6Ad"
                    "V6Av"
                    "VIP"
                )
            else
                declare -a arr=(${ROI})
            fi

            for roi in "${arr[@]}"; do
                echo ${roi}
                
                mkdir -p ${sf}/mROI/${roi}
                mkdir -p ${ROI_FLD}/1mm
                
                # resample roi to 1 mm
                mri_convert ${ROI_FLD}/${roi}.nii.gz ${ROI_FLD}/1mm/${roi}.nii.gz -vs 1 1 1
                
                fslmaths ${ROI_FLD}/1mm/${roi}.nii.gz -sub 1 -mul 99 ${ROI_FLD}/1mm/${roi}_SUB.nii.gz
                # mask with roi file
                for file in ${sf}/*.nii.gz; do
                    #echo Masking ${file}
                    #echo With ${ROI_FLD}/1mm/${roi}.nii.gz
                    fslmaths ${file} -mas ${ROI_FLD}/1mm/${roi}.nii.gz ${sf}/mROI/${roi}/$(basename "$file")
                    fslmaths ${sf}/mROI/${roi}/$(basename "$file") -add ${ROI_FLD}/1mm/${roi}_SUB.nii.gz \
                             ${sf}/mROI/${roi}/$(basename "$file")
                done
            done
        fi
    done
}

In [8]:
# cycle over monkeys, models, and R2-thresholds
declare -a ROIs=('V1' 'V2' 'V3d' 'V3v' 'V3A' 'V4' 'V4t' 'V4v' 'MT' 'MST' 
    'TEO' 'TAa' 'Tpt' 'TPO' 'FST' 'VIP' 'LIPv' 'LIPd' 
    '1-2' 'PL' 'PM' 'LGNm' 'LGNp' 'Striatum' '3a_b' 'SII' 
    'F2_(6DR_6DC)' 'F4' 'F5_(6Va_6Vb)' 'F7' '8Av' '8Ad' '8Bs' '8Bd' '46v' '46d' 
    '12r' '12m' '13l' 'Id' 'Iai' '32' '24c' 'ML' '10mr')

for M in "${Monkeys[@]}"; do
    for S in "${Models[@]}"; do
        for R in "${ROIs[@]}"; do
            MaskResults_inAnat_withROI $M $S $R $FitRes
        done
    done
done  

Processing monkey danny
csshrf_cv1_dhrf V1
V1
mri_convert /Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults/Reference/Volumes/atlas/danny/ROI_manualadjust/V1.nii.gz /Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults/Reference/Volumes/atlas/danny/ROI_manualadjust/1mm/V1.nii.gz -vs 1 1 1 
reading from /Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults/Reference/Volumes/atlas/danny/ROI_manualadjust/V1.nii.gz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (1, 0, 0)
j_ras = (0, 1, 0)
k_ras = (0, 0, 1)
Reslicing using trilinear interpolation 
writing to /Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults/Reference/Volumes/atlas/danny/ROI_manualadjust/1mm/V1.nii.gz...
V1
mri_convert /Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults/Reference/Volumes/atlas/danny/ROI_manualadjust/V1.nii.gz /Users/chris/Documents/MRI_ANALYSIS/NHP-PRF/FitResults/Reference/Volumes/atlas/danny/ROI_manualadjust/1mm/V1.nii.gz -vs 1 1 1 
reading from /Users/chris/Documents/MRI_ANALYSIS/NHP-